# Universidad del Valle de Guatemala
# NLP
# Andres Quezada 21085
# Laboratorio 1 – Anonimización de PII con Expresiones Regulares

## Paso 1 – Imports y carga del dataset

In [80]:
import pandas as pd, re, hashlib
DATA_PATH = './data/pii_dataset.csv'
df = pd.read_csv(DATA_PATH)
df.head()

,Nombre,Correo,Teléfono,Dirección,Identificación,Texto
0,Reyes Pinilla Rosado,woliver@tena.es,+34967 400 699,"Calle Rambla de Nieves Dalmau, 126, Ávila",40955969R,"Mi nombre es Reyes Pinilla Rosado, puedes cont..."
1,Evita Solsona Escobar,nicolaspou@hernandez.com,991740961,"Calle Vial Luís Martín, 131, Las Palmas",34970246V,"Mi nombre es Evita Solsona Escobar, puedes con..."
2,Lorenza Mate Bayón,jimena05@iglesias.com,+34 673931008,"Alameda Amador Calleja 13 Piso 1 , León, 91297",P5293902,"Mi nombre es Lorenza Mate Bayón, puedes contac..."
3,Renato González-Araujo,tomasalondra@higueras.es,916543174,"Piso 5, Ávila",Y6585174Z,"Mi nombre es Renato González-Araujo, puedes co..."
4,Chus Alejandro Duarte Mayoral,iferrando@peiro.es,+34 650500564,"Urbanización Eliseo Naranjo 68 Piso 8 , Ciudad...",P4418638,"Mi nombre es Chus Alejandro Duarte Mayoral, pu..."


## Paso 2 – Exploración inicial

In [81]:
print(f'Filas: {df.shape[0]}, Columnas: {df.shape[1]}')
df.info()

Filas: 100, Columnas: 6
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Nombre          100 non-null    object
 1   Correo          100 non-null    object
 2   Teléfono        100 non-null    object
 3   Dirección       100 non-null    object
 4   Identificación  100 non-null    object
 5   Texto           100 non-null    object
dtypes: object(6)
memory usage: 4.8+ KB


## Paso 3 – Definir patrones regex para PII

In [82]:
patterns = {
    'EMAIL': re.compile(r"\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b", re.I),
    'PHONE': re.compile(r"(?:\+34[\s\-]?)?(?:\(\d{3}\)[\s\-]?\d{3}[\s\-]?\d{3}|\d{3}[\s\-]?\d{3}[\s\-]?\d{3}|\d{3}[\s\-]?\d{2}[\s\-]?\d{2}[\s\-]?\d{2})"),
    'ID':    re.compile(r"\b(?:\d{8}[A-Z]|[XYZ]\d{7}[A-Z]|[A-Z]\d{7})\b", re.I),
    'ADDRESS': re.compile(r"(?i)\b(calle|paseo|avenida|urb\.?|urbanización|piso|ciudad|camino|plaza|puerta|callejón|vial|apt|pasaje)\b.*?\d+"),
    'NAME': re.compile(r"\b(?:[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+(?:\s+|[-])){1,}[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+\b")
}
patterns


{'EMAIL': re.compile(r'\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b',
            re.IGNORECASE|re.UNICODE),
 'PHONE': re.compile(r'(?:\+34[\s\-]?)?(?:\(\d{3}\)[\s\-]?\d{3}[\s\-]?\d{3}|\d{3}[\s\-]?\d{3}[\s\-]?\d{3}|\d{3}[\s\-]?\d{2}[\s\-]?\d{2}[\s\-]?\d{2})',
            re.UNICODE),
 'ID': re.compile(r'\b(?:\d{8}[A-Z]|[XYZ]\d{7}[A-Z]|[A-Z]\d{7})\b',
            re.IGNORECASE|re.UNICODE),
 'ADDRESS': re.compile(r'(?i)\b(calle|paseo|avenida|urb\.?|urbanización|piso|ciudad|camino|plaza|puerta|callejón|vial|apt|pasaje)\b.*?\d+',
            re.IGNORECASE|re.UNICODE),
 'NAME': re.compile(r'\b(?:[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+(?:\s+|[-])){1,}[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+\b',
            re.UNICODE)}

## Paso 4 – Función de ofuscación

In [83]:
def _token(tag: str, original: str) -> str:
    return f"<{tag}:{hashlib.sha1(original.encode()).hexdigest()[:6]}>"

def redact(text):
    if not isinstance(text, str):
        return text
    for tag, pat in patterns.items():
        text = pat.sub(lambda m: _token(tag, m.group(0)), text)
    return text

## Paso 5 – Aplicar ofuscación

In [84]:
df_clean = df.applymap(redact)
df_clean.head()

/var/folders/k7/tjmwvmc549lftb1qckjbd96m0000gn/T/ipykernel_40521/1788964534.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df.applymap(redact)


,Nombre,Correo,Teléfono,Dirección,Identificación,Texto
0,<NAME:be0fa2>,<EMAIL:5fceaa>,<PHONE:90fad4>,"<ADDRESS:8b0e79>, Ávila",<ID:868fc9>,"Mi nombre es <NAME:be0fa2>, puedes contactarme..."
1,<NAME:3360e8>,<EMAIL:70b8ea>,<PHONE:561259>,"<ADDRESS:afc25f>, <NAME:65dd75>",<ID:5c44da>,"Mi nombre es <NAME:3360e8>, puedes contactarme..."
2,<NAME:eb1a10>,<EMAIL:ac43ac>,<PHONE:8c224d>,"<NAME:7c2b8f> 13 <ADDRESS:510a4a> , León, 91297",<ID:ad8cf7>,"Mi nombre es <NAME:eb1a10>, puedes contactarme..."
3,<NAME:ca1fcc>,<EMAIL:a88c0f>,<PHONE:fda581>,"<ADDRESS:9fefee>, Ávila",<ID:a64f59>,"Mi nombre es <NAME:ca1fcc>, puedes contactarme..."
4,<NAME:fa3ad7>,<EMAIL:890d19>,<PHONE:cb8ec1>,"<ADDRESS:379cef> <ADDRESS:12ec9c> , <ADDRESS:8...",<ID:7fd888>,"Mi nombre es <NAME:fa3ad7>, puedes contactarme..."


## Paso 6 – Contar reemplazos

In [85]:
from collections import Counter
counts = Counter()
for col in df.columns:
    for val in df[col]:
        if isinstance(val, str):
            for tag, pat in patterns.items():
                counts[tag] += len(pat.findall(val))
counts

Counter({'NAME': 362, 'ADDRESS': 205, 'EMAIL': 200, 'PHONE': 200, 'ID': 200})

## Paso 7 – Guardar dataset limpio

In [86]:
OUTPUT_PATH = 'pii_dataset_clean.csv'
df_clean.to_csv(OUTPUT_PATH, index=False)
print(f'Dataset limpio guardado en {OUTPUT_PATH}')

Dataset limpio guardado en pii_dataset_clean.csv


## Paso 8 – Conclusiones y mejoras
- **Limitaciones:** Regex no detecta formatos poco comunes; no cubre direcciones al total, ni nombres.
- **Mejoras:** Añadir NER con spaCy, validar DNIs, ampliar patrones.
